In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Myriad Pro'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600

import numpy as np
import scipy.stats as stats

from scipy.stats import kstest, cramervonmises
import tensorflow as tf
import tensorflow_probability as tfp
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import time

from ipcc_colormap import *
from utils import *

from matplotlib import gridspec
from matplotlib.colorbar import ColorbarBase

import pickle

coastline = gpd.read_file('/home/mizu_home/xp53/nas/home/coastlines-split-SGregion/lines.shp')
mask = np.loadtxt('mask.txt')

ipcc_blue = (112.0/255, 160.0/255, 205.0/255, 1.0)
ipcc_orange = (196.0/255, 121.0/255, 0.0/255, 1.0)

tmp_cmap = ipcc_cmap()
tmp_cmap.read_rgb_data_from_excel()
;

2024-09-05 16:21:59.711702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 16:21:59.806054: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


''

In [2]:
# DATA PREPARATION
# the station-based rainfall data is already organized in a N by P matrix
rain_obs = np.loadtxt('data/sta_monthly.csv')

# the simulated rainfall data is reshaped to a 2d matrix of size N by (W x L)
rain_sim_flatten = np.loadtxt('data/wrf_monthly.csv')
rain_sim = rain_sim_flatten.reshape(rain_sim_flatten.shape[0], 120, 160)

# sim_sel constains a P by 4 matrix
# the first two columns are the row and column indices of grids corresponding to the stations
# the last two columns are the lons and lats of grids corresponding to the stations
sim_sel = np.loadtxt('data/wrf_loc.csv')
sim_idx = sim_sel[:, :2].astype(int)

# select simulated rainfall at stations
wrf_sta = np.array([rain_sim[:, i, j] for (i, j) in sim_idx]).T

# lats and lons of all grids are stored in a 2d matrix of 2 by (W x L)
# the first row is lons and the second row is lats
longlat = np.loadtxt('data/lonlat.txt')
lons = longlat[0, :].reshape(120, 160)
lats = longlat[1, :].reshape(120, 160)

# read station lons (3rd column) and lats (4th column)
sta_loc = np.genfromtxt('data/sta_lookup_new.csv', delimiter=',')[:, 2:]

# read wrf simulated rainfall forced by cmip6 future climate
cmip_rain_f = np.loadtxt('data/wrf_monthly_cmip.txt')[-40:, :]
cmip_rain0 = cmip_rain_f.reshape(cmip_rain_f.shape[0], 12, 120, 160)
cmip_rain = np.zeros(cmip_rain0.shape)
cmip_rain[:, 0:6, :, :] = cmip_rain0[:, 6:12, :, :]
cmip_rain[:, 6:12, :, :] = cmip_rain0[:, 0:6, :, :]
cmip_rain_flatten = np.reshape(cmip_rain, (40, 12, 120*160))
wrf_sta_fut = np.zeros((40, 12, sim_idx.shape[0]))
for idx, (i, j) in enumerate(sim_idx):
    wrf_sta_fut[:, :, idx] = cmip_rain[:, :, i, j]


In [10]:
# interpolate future rainfall had the same obervations been made at the stations
sub_name = ['(a) Jan', '(b) Feb', '(c) Mar', '(d) Apr', '(e) May', '(f) Jun', 
            '(g) Jul', '(h) Aug', '(i) Sep', '(j) Oct', '(k) Nov', '(l) Dec']
pkl_file = open('his_rain_interpolations_twostage.pkl', 'rb')
rain_h = pickle.load(pkl_file)
gpr_ = gp_interpolator(P = sim_idx.shape[0])
fig, ax = plt.subplots(nrows = 4, ncols = 3, figsize = (9.5, 12))

for i in range(12):
    t1 = time.time()
    gpr_.read_rainfall(rain_obs[i::12, :], wrf_sta_fut[:, i, :])
    gpr_.sn_converge()
    r1, _ = gpr_.predict(cmip_rain_flatten[:, i, :])
    r1 = np.reshape(r1.T, (40, 120, 160))

    ridx, cidx = divmod(i, 3) 
    tmp_obs = np.mean(rain_obs[i::12, :], axis = 1)
    tmp_rf = np.nanmean(np.multiply(r1, mask), axis = (1, 2))
    tmp_rh = np.nanmean(np.multiply(rain_h[i::12,:,:], mask), axis = (1, 2))
    ax[ridx][cidx].scatter(tmp_obs, tmp_rh, s = 40, color = ipcc_blue, label = 'ERA5', alpha = 0.67, edgecolors = 'None')
    ax[ridx][cidx].scatter(tmp_obs, tmp_rf, s = 40, color = ipcc_orange, label = 'CMIP6', alpha = 0.67, edgecolors = 'None')
    ax[ridx][cidx].axline([0, 0], [1, 1], color = 'black', linestyle = '--')
    print('month %d takes %f seconds' % (i, time.time() - t1))
    x_min, x_max = np.min(tmp_obs), np.max(tmp_obs)
    ax[ridx][cidx].set_xlim([np.max([0, x_min - 20]), x_max + 20])
    ax[ridx][cidx].set_ylim([np.max([0, x_min - 20]), x_max + 20])
    # set tick gap to 100
    ax[ridx][cidx].xaxis.set_major_locator(mticker.MultipleLocator(100))
    ax[ridx][cidx].yaxis.set_major_locator(mticker.MultipleLocator(100))
    # use a square plot
    ax[ridx][cidx].set_aspect('equal', adjustable='box')
    ax[ridx][cidx].text(0.05, 0.95, f'{sub_name[i]}', transform=ax[ridx][cidx].transAxes, 
                         fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.6))

    if i == 11:
        ax[ridx][cidx].legend(loc = 'lower right')
fig.text(0.5, 0.04, 'Arithmetic Average of Rainfall [mm]', ha='center', fontsize=14)
fig.text(0.04, 0.5, 'Spatial Average of Rainfall [mm]', va='center', rotation='vertical', fontsize=14)

# Adjust layout and save the figure
fig.tight_layout(rect=[0.05, 0.05, 1, 1])  # Adjust layout for space around text
fig.savefig('figures_two_stage/fut_his_comp_twostage.png', dpi = 600, bbox_inches = 'tight')


month 0 takes 17.700406 seconds
month 1 takes 17.508697 seconds
month 2 takes 17.804149 seconds
month 3 takes 17.089255 seconds
month 4 takes 17.290958 seconds
month 5 takes 17.286037 seconds
month 6 takes 17.292889 seconds
month 7 takes 17.200429 seconds
month 8 takes 17.264842 seconds
month 9 takes 17.446227 seconds
month 10 takes 17.308879 seconds
month 11 takes 17.113224 seconds
